In [2]:
# -*- coding: utf-8 -*-
# @Time    : 2022/3/30 20:43
# @Author  : huangkai
# @File    : search_content.py


import sys

sys.path.append('../recall/nlp4es8/ir/')
sys.path.append('../utils')

from es_config import Config


# from utils.logger_config import base_logger
# from utils.args import FLAGS
# from ir.index_hnsw import Index


class Search(object):
    def __init__(self, config, index_name):
        # base_logger.info("Searching ...")
        self.config = config
        self.es = self.config.es

        self.index_name = index_name

    def searchAnswer(self, question, ):
        body = {
            "query": {
                "multi_match": {
                    "query": question,
                    "fields": ["document"],  # 在question字段中匹配查询
                    "type": "most_fields",
                }
            }
        }

        # es相关配置

        res = self.es.search(index=self.index_name, body=body, request_timeout=30, size=self.config.top_n)

        topn = res['hits']['hits']

        result = []
        for data in topn:
            result.append(
                (
                    data['_source']['index'],
                    data['_source']['document']
                )
            )
        return result

        # base_logger.info("ReadTimeOutError may not be covered!")



In [16]:
import pandas as pd
import json

index_file_content_config = Config()
index_file_content_config.index_name = "kbqa"
index_file_content_config.top_n=1
search_name = Search(index_file_content_config, "kbqa")



In [4]:
while True:
    query = input("please input")
    result = search_name.searchAnswer(query)
    print(result[0])
    for data in result:
        print("question:%s  question_id:%s  " % (data[0], data[1]))

please input足球运动员莫侯斯所在的城市是哪个
('6068036', '莫侯斯 ||| 运动项目 ||| 足球')
question:6068036  question_id:莫侯斯 ||| 运动项目 ||| 足球  
question:6068038  question_id:莫侯斯 ||| 所在城市 ||| 米盧斯  
question:7483668  question_id:侯斯特 ||| 职业 ||| 运动员 足球  
question:26744282  question_id:拉莫斯(洪都拉斯足球运动员) ||| 运动项目 ||| 足球  
question:1070692  question_id:莫里斯(足球运动员) ||| 别名 ||| 莫里斯  
question:32913968  question_id:斯莫德斯 ||| 职业 ||| 运动员 足球  
question:10205418  question_id:纳塞夫·莫里斯(足球运动员) ||| 运动项目 ||| 足球  
question:8946580  question_id:霍莫斯 ||| 职业 ||| 运动员 足球  
question:9088935  question_id:迪莫斯 ||| 职业 ||| 运动员 足球  
question:40998095  question_id:夏莫斯 ||| 职业 ||| 运动员 足球  
question:19996452  question_id:斯莫拉 ||| 职业 ||| 运动员 足球  
question:39176369  question_id:巴莫斯 ||| 职业 ||| 运动员 足球  
question:41182628  question_id:拉莫斯 ||| 职业 ||| 足球运动员  
question:16616359  question_id:莫斯达 ||| 职业 ||| 运动员 足球  
question:25987545  question_id:阿斯莫 ||| 职业 ||| 运动员 足球  
question:23974383  question_id:莫伊斯 ||| 职业 ||| 运动员 足球  
question:25779348  question_id:苏莫斯 ||| 职业 |

KeyboardInterrupt: Interrupted by user

In [48]:
import json
data=json.loads(open("es_llm_table_tuple_top20.json",encoding="utf8").read())

In [23]:
unit

{'id': 'TE001643',
 'question': '“互联网上充斥着各种娱乐信息，今天我意外地发现了一个音乐乐队——冷血动物乐队，对于它来说，我完全是个外行，所以我想了解更多关于这个乐队的信息，例如它的中文名、成立时间、曲目、唱片公司还有他们的成员。”',
 'attribute': ['冷血动物乐队 ||| 中文名 ||| 冷血动物乐队',
  '冷血动物乐队 ||| 成立时间 ||| 1994年',
  '冷血动物乐队 ||| 音乐类型 ||| 摇滚',
  '冷血动物乐队 ||| 类型 ||| 地下摇滚劲旅',
  '冷血动物乐队 ||| 组合形式 ||| 三人组合',
  '冷血动物乐队 ||| 唱片公司 ||| 骏驎唱片',
  '冷血动物乐队 ||| 成员 ||| 谢天笑',
  '李明',
  '梁旭',
  '']}

In [49]:
from tqdm import tqdm
count=0
for unit in data[:-13]:
    temp_attr=[]
    for attr in unit["attribute"]:
        if "没有找到" not in attr:
            result = search_name.searchAnswer(attr)[0][1]
            if result!=attr:
                count+=1
                print(result,"===",attr)

Amy Klinprathum ||| 别名 ||| Amy Klinprathum === Amy Klinprathum ||| 外文名 ||| Amy Klinprathum;Amy Klinprathum ||| 别名 ||| Amy Klinprathum;Amy Klinprathum ||| 昵称 ||| Amy;Amy Klinprathum ||| 出生地 ||| 美国
Amy Klinprathum ||| 配偶 ||| Cee<Siwat Chotechaicharin> === Amy Klinprathum ||| 配偶 ||| Cee<Siwat Chotechaicharin
巴托洛米奥 ||| 悬赏金额 ||| 1亿5000万贝里（漫画706提到） === 巴托洛米奥 ||| 悬赏金 ||| 1亿5000万贝里(漫画706提到)
哈塞克 ||| 人口（2007年） ||| 81,809 === 哈塞克 ||| 人口(2007年) ||| 81,809
少年祖师 ||| 作者 ||| 不忘订阅 === 少年祖师爷 ||| 作者 ||| 不忘订阅
千手柱间 ||| 想挑战的对手 ||| 宇智波斑 === 宇智波斑 ||| 想挑战的对手 ||| 千手柱间
干毛毛 ||| 出生日期 ||| 1989年2月21号 === 干毛毛 ||| 出生日期 ||| 不记
干毛毛 ||| 国 籍 ||| 中国 === 干毛毛 ||| 国籍 ||| 中国
刘少奇 ||| 副主席 ||| 董必武、宋庆龄（共同担任） === 刘少奇 ||| 副主席 ||| 董必武、宋庆龄(共同担任)
瑞根 ||| 爱好 ||| 阅读，旅游 === 柴碧云 ||| 爱好 ||| 旅游、阅读
奥斯汀 ||| 人口（2006年） ||| 856,254 === 奥斯汀 ||| 人口(2006年) ||| 856,254
永璇 ||| 名 ||| 永璇 === 永璇 ||| 姓名 ||| 永璇


KeyboardInterrupt: 

In [28]:
count

84

In [47]:
count=0
for unit in data[:-13]:
    temp_attr=[]
    for attr in unit["attribute"]:
        if attr:
            if "没有找到" not in attr:
                if ";" in attr:
                    attr=attr.split(";")
                    for a in attr:
                        if a:
                            result = search_name.searchAnswer(a)[0][1]
                            temp_attr.append(result)
                else:
                    result = search_name.searchAnswer(attr)[0][1]
                    temp_attr.append(result)
            else:
                temp_attr.append(attr)
    unit["attribute"] = temp_attr

KeyboardInterrupt: 

In [40]:
 unit["attribute"]

['汪小敏 ||| 喜欢的电视 ||| 《吸血鬼日记》']

In [34]:
unit["attribute"]

['冷血动物乐队 ||| 中文名 ||| 冷血动物乐队',
 '冷血动物乐队 ||| 成立时间 ||| 1994年',
 '冷血动物乐队 ||| 音乐类型 ||| 摇滚',
 '冷血动物乐队 ||| 类型 ||| 地下摇滚劲旅',
 '冷血动物乐队 ||| 组合形式 ||| 三人组合',
 '冷血动物乐队 ||| 唱片公司 ||| 骏驎唱片',
 '冷血动物乐队 ||| 成员 ||| 谢天笑',
 '李明',
 '梁旭',
 '']

In [14]:
count/len(data[:-13])

0.224

In [15]:
attr

'汪小敏 ||| 喜欢的电视 ||| 《吸血鬼日记》'

In [42]:
submission = open("es_llm_table_tuple_top20_post_process.json", "w", encoding="utf8")

submission.write(json.dumps(data, ensure_ascii=False))

293743

In [45]:
data[2]

{'id': 'TE001070',
 'question': '哪些官职在松平赖桓的身上',
 'attribute': ['松平赖桓 ||| 朝廷官位 ||| 从四位上、讃岐守、侍从']}

In [51]:
st='霸州市新利钢铁有限公司 ||| 别名 ||| 霸州市新利钢铁有限公司'
str(set(st))

"{'钢', '限', '利', '|', ' ', '新', '司', '州', '名', '市', '霸', '别', '公', '有', '铁'}"